In [1]:
import os
import tensorflow as tf
import tensorflow_recommenders as tfrs
import json
import pandas as pd
from os import getenv
import numpy as np
#from keras import models
from firebase_admin import credentials, firestore, initialize_app

In [2]:
cred = credentials.Certificate("key.json")
default_app = initialize_app(cred)
db = firestore.client()

In [3]:
class User(object):
    def __init__(self, USER_ID, Nama, Gender, Age, Tipe_Masalah):
        self.USER_ID = USER_ID
        self.Nama = Nama
        self.Gender = Gender
        self.Age = Age
        self.Tipe_Masalah = Tipe_Masalah

    @staticmethod
    def from_json_do_not_use(source):
        """
        Source is expected from pure json string
        """
        return source

    @staticmethod
    def from_dict(source):
        # [START_EXCLUDE]
        user = User(source[u'USER_ID'], source[u'Nama'], source[u'Gender'], source[u'Age'], source[u'Tipe_Masalah'])

        if u'USER_ID' in source:
            user.USER_ID = source[u'USER_ID']

        if u'Nama' in source:
            user.Nama = source[u'Nama']

        if u'Gender' in source:
            user.Gender = source[u'Gender']

        if u'Age' in source:
            user.Age = source[u'Age']

        if u'Tipe_Masalah' in source:
            user.Tipe_Masalah = source[u'Tipe_Masalah']

        return user
        # [END_EXCLUDE]

    def to_dict(self):
        # [START_EXCLUDE]
        dest = {
            u'USER_ID': self.USER_ID,
            u'Nama': self.Nama,
            u'Gender': self.Gender,
            u'Age': self.Age,
            u'Tipe_Masalah': self.Tipe_Masalah
        }

        if self.USER_ID:
            dest[u'USER_ID'] = self.USER_ID

        if self.Nama:
            dest[u'Nama'] = self.Nama

        if self.Gender:
            dest[u'Gender'] = self.Gender

        if self.Age:
            dest[u'Age'] = self.Age

        if self.Tipe_Masalah:
            dest[u'Tipe_Masalah'] = self.Tipe_Masalah

        return dest
        # [END_EXCLUDE]

    
    def __repr__(self):
        return (
            f'City(\
                USER_ID={self.USER_ID}, \
                Nama={self.Nama}, \
                Age={self.Age}, \
                Gender={self.Gender}, \
                Tipe_Masalah={self.Tipe_Masalah}\
            )'
        )

In [4]:
zidan = User(1, "zidan", "pria", 43, "Depresi")

In [5]:
zidan

City(                USER_ID=1,                 Nama=zidan,                 Age=43,                 Gender=pria,                 Tipe_Masalah=Depresi            )

In [6]:
type(zidan)

__main__.User

In [7]:
type(zidan.to_dict())

dict

In [8]:
import pathlib
import os

user_file = "\\Data\\Processed User Data.json"
caregiver_file = "\\Data\\Processed Caregiver Data.json"

path = pathlib.Path().resolve()
parent_path = path.parent.absolute()

user_path = str(parent_path) + user_file
caregiver_ = str(parent_path) + caregiver_file

In [9]:
"""
with open(user_path) as file:
    data = json.load(file)
    db_user_placeholder = db.collection(u'User_Data_Placeholder')
    for user_dict in data:
        user_dict['Nama'] = 'placeholder_nama'
        user_dict['Text'] = 'Placeholder_text'
        db_user_placeholder.add(user_dict)
"""

"\nwith open(user_path) as file:\n    data = json.load(file)\n    db_user_placeholder = db.collection(u'User_Data_Placeholder')\n    for user_dict in data:\n        user_dict['Nama'] = 'placeholder_nama'\n        user_dict['Text'] = 'Placeholder_text'\n        db_user_placeholder.add(user_dict)\n"

In [10]:
data = db.collection(u'user_pref').get()


In [11]:
print(data[0].to_dict())

{'gender': 'Pria', 'birthday': DatetimeWithNanoseconds(2000, 11, 16, 17, 0, tzinfo=datetime.timezone.utc), 'text': 'placeholder text', 'problems': 'Depresi Gangguan-makan'}


In [12]:
def convert_snapshot(data, desired_key = None):
    """
    this extract desired keys from list of dicts from snapshot
    """
    data_dictionary = map(lambda x: x.to_dict(), data)
    if (desired_key is not None):
        data_dictionary = map(lambda x: dict((k, x[k]) for k in desired_key if k in x), list(data_dictionary))

    return list(data_dictionary)

In [13]:
new_data = convert_snapshot(data, ["gender","problems","age","text"])

In [14]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    """
    A utility method to create a tf.data dataset from a Pandas Dataframe
    """
    dataframe = dataframe.copy()
    ds = tf.data.Dataset.from_tensor_slices(dict(dataframe))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [15]:
def convert_categorical_data(df, col, is_caregiver = False):
    """
    Convert categorical data that splited by space to their own column in pandas dataframe , making them like one hot encoded
    """
    import numpy as np
    ### Join every string in every row, split the result, pull out the unique values.
    genres = np.unique(' '.join(df[col]).split(' '))
    ### Drop 'NA'
    genres = np.delete(genres, np.where(genres == ''))
    if(not is_caregiver):
        for genre in genres:
            df[genre] = df[col].str.contains(genre).astype('int')
    else:
        for genre in genres:
            df['Caregiver-'+genre] = df[col].str.contains(genre).astype('int')
    df.drop(col, axis=1, inplace=True)

In [16]:
def fill_non_existent_column(df, is_caregiver = False):
    """
    This fill the df with unique problems column i
    """
    if is_caregiver is False:
         new_column = ['ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan','Gangguan-stres-pascatrauma', 'Skizofrenia']
    else:
        new_column = ['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan','Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']

    for col in new_column:
        if(col in df.columns):
            pass
        else:
            df[col] = 0
    
    

In [17]:
def change_column_name(df, old_name,new_name):
    """
    replace list of column name in old_name to new colum name in new_name
    the order of old_name and new_name need to be exact
    """
    res = {old_name[i]: new_name[i] for i in range(len(new_name))}
    df.rename(columns = res, inplace = True)
    
    

In [18]:
def desired_keys(dictionary , desired_key):
    if (desired_key is not None):
        dictionary = map(lambda x: dict((k, x[k]) for k in desired_key if k in x), list(dictionary))
    return list(dictionary)

In [19]:
#ds_kalimat = tf.data.Dataset.from_tensor_slices(pd.DataFrame.from_dict(new_data).to_dict(orient="list"))

In [20]:
user_df = pd.DataFrame.from_dict(new_data)

In [21]:
user_df.head(4)

,gender,problems,text,age
0,Pria,Depresi Gangguan-makan,placeholder text,NaN
1,Laki-laki,Gangguan-kecemasan Depresi,Placeholder_text,27.0


In [22]:
convert_categorical_data(user_df, col='problems')

In [23]:
user_df.head(4)

,gender,text,age,Depresi,Gangguan-kecemasan,Gangguan-makan
0,Pria,placeholder text,NaN,1,0,1
1,Laki-laki,Placeholder_text,27.0,1,1,0


In [24]:
change_column_name(user_df, ['gender','age'], ['Gender','Age'])

In [25]:
user_df.head(4)

,Gender,text,Age,Depresi,Gangguan-kecemasan,Gangguan-makan
0,Pria,placeholder text,NaN,1,0,1
1,Laki-laki,Placeholder_text,27.0,1,1,0


In [26]:
user_df.columns

Index(['Gender', 'text', 'Age', 'Depresi', 'Gangguan-kecemasan',
       'Gangguan-makan'],
      dtype='object')

In [27]:
 fill_non_existent_column(user_df)

In [28]:
user_df.head(4)


,Gender,text,Age,Depresi,Gangguan-kecemasan,Gangguan-makan,ADHD-Hiperaktif-dan-kurang-fokus,Gangguan-stres-pascatrauma,Skizofrenia
0,Pria,placeholder text,NaN,1,0,1,0,0,0
1,Laki-laki,Placeholder_text,27.0,1,1,0,0,0,0


In [29]:
user_ds = df_to_dataset(user_df)

In [30]:
#caregiver_query

In [31]:
data = db.collection(u'chat_rooms').get()

In [32]:
from datetime import date
 
def age(birthdate):
    today = date.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

In [33]:
import flatdict

In [34]:

def Merge(dict1, dict2):
    """
    Python code to merge dict
    """
    res = {**dict1, **dict2}
    return res
    

def unpack_caregiver_dict(dictionary):
    return 0

In [35]:
def unpack_caregiver_snapshot(data):
    desired_keys = ["problems", "birthday", "gender","caregiver_id","text"]
    #this unpack the value of snapshot list and turn the snapshot into dictionary also add aditional key for caregiver_id
    caregiver_dict_unpacked = map(lambda feature: Merge(feature.to_dict(), {'caregiver_id':feature.id}) , data)
    
    #this unpack the snapshot in 'caregiver' snapshot and add them back to main dictionary
    caregiver_dict = map(lambda feature: Merge(feature , feature.pop('caregiver').get().to_dict()) , caregiver_dict_unpacked )
    
    #this seperate desired keys from unused keys
    caregiver_dict = map(lambda x: dict((k, x[k]) for k in desired_keys if k in x), caregiver_dict)
    
    #this convert birthday stamp into interger age
    caregiver_dict = map(lambda feature: Merge(feature , {'age':age(feature.pop('birthday'))}), caregiver_dict)
    
    return list(caregiver_dict)

In [36]:
caregiver_dict = unpack_caregiver_snapshot(data)

In [37]:
caregiver_dict

[{'problems': 'Depresi Gangguan-makan',
  'gender': 'Pria',
  'caregiver_id': 'HUDwzgELY1OEzwieoHXK',
  'text': 'Susah makan gara-gara diputusin pacar',
  'age': 21}]

In [38]:
def convert_caregiver_dictList_to_df(dictionary):
    # Creates DataFrame. 
    df = pd.DataFrame(dictionary) 
    #change column name to feature according to the model
    change_column_name(df, ['problems','gender','caregiver_id','age'],['Caregiver_Tipe_Masalah','Caregiver_Gender','CAREGIVER_ID','Caregiver_Age'])
    #convert categorical data into their own column
    convert_categorical_data(df, "Caregiver_Tipe_Masalah", is_caregiver = True)
    #fill missing column with default value
    fill_non_existent_column(df, is_caregiver = True)
    return df

In [39]:
# Creates DataFrame. 
df = convert_caregiver_dictList_to_df(caregiver_dict) 

In [40]:
caregiver_ds = df_to_dataset(df)

In [41]:
for feature_batch in caregiver_ds:
  print('Every feature:', list(feature_batch.keys()))

Every feature: ['Caregiver_Gender', 'CAREGIVER_ID', 'text', 'Caregiver_Age', 'Caregiver-Depresi', 'Caregiver-Gangguan-makan', 'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']


In [42]:
df

,Caregiver_Gender,CAREGIVER_ID,text,Caregiver_Age,Caregiver-Depresi,Caregiver-Gangguan-makan,Caregiver-ADHD-Hiperaktif-dan-kurang-fokus,Caregiver-Gangguan-kecemasan,Caregiver-Gangguan-stres-pascatrauma,Caregiver-Skizofrenia
0,Pria,HUDwzgELY1OEzwieoHXK,Susah makan gara-gara diputusin pacar,21,1,1,0,0,0,0


In [43]:
caregiver_index = df.to_dict('records')

In [44]:
caregiver_index

[{'Caregiver_Gender': 'Pria',
  'CAREGIVER_ID': 'HUDwzgELY1OEzwieoHXK',
  'text': 'Susah makan gara-gara diputusin pacar',
  'Caregiver_Age': 21,
  'Caregiver-Depresi': 1,
  'Caregiver-Gangguan-makan': 1,
  'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus': 0,
  'Caregiver-Gangguan-kecemasan': 0,
  'Caregiver-Gangguan-stres-pascatrauma': 0,
  'Caregiver-Skizofrenia': 0}]

In [45]:
caregiver_index = map(lambda x: dict((k, [x[k]]) for k in x), caregiver_index)

In [46]:
list(caregiver_index)

[{'Caregiver_Gender': ['Pria'],
  'CAREGIVER_ID': ['HUDwzgELY1OEzwieoHXK'],
  'text': ['Susah makan gara-gara diputusin pacar'],
  'Caregiver_Age': [21],
  'Caregiver-Depresi': [1],
  'Caregiver-Gangguan-makan': [1],
  'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus': [0],
  'Caregiver-Gangguan-kecemasan': [0],
  'Caregiver-Gangguan-stres-pascatrauma': [0],
  'Caregiver-Skizofrenia': [0]}]

In [47]:
MAIN_TEXT_MODEL =  tf.keras.models.load_model('saved_manual_model/text_query_v1')
MAIN_USER_MODEL =   tf.keras.models.load_model('saved_manual_model/user_query_v1')
MAIN_CAREGIVER_MODEL =  tf.keras.models.load_model('saved_manual_model/caregiver_query_v1')

In [48]:
caregiver_index = caregiver_ds.map(lambda features: (features['CAREGIVER_ID'], MAIN_CAREGIVER_MODEL(features)))

In [49]:
brute_force = tfrs.layers.factorized_top_k.BruteForce(MAIN_USER_MODEL)
brute_force.index_from_dataset(  
       caregiver_ds.map(lambda features: (features['CAREGIVER_ID'], MAIN_CAREGIVER_MODEL(features)))
)





In [50]:
a =['Gender', 'Age', 'ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan','Gangguan-stres-pascatrauma', 'Skizofrenia']
value = ['Pria', 30, 0, 1, 0, 0, 1, 0]
dictionary_Test = dict((el,([value[ix]])) for ix,el in enumerate(a))
print(dictionary_Test)

{'Gender': ['Pria'], 'Age': [30], 'ADHD-Hiperaktif-dan-kurang-fokus': [0], 'Depresi': [1], 'Gangguan-kecemasan': [0], 'Gangguan-makan': [0], 'Gangguan-stres-pascatrauma': [1], 'Skizofrenia': [0]}


In [52]:

a, titles = brute_force(dictionary_Test, k=1)
print(f"Top recommendations: {titles[0]}")

Top recommendations: [b'HUDwzgELY1OEzwieoHXK']


In [53]:
brute_force = tfrs.layers.factorized_top_k.BruteForce(MAIN_TEXT_MODEL)
brute_force.index_from_dataset(  
       caregiver_ds.map(lambda features: (features['CAREGIVER_ID'], MAIN_TEXT_MODEL(features['text'])))
)


In [55]:

a, titles = brute_force(np.array(["halo aku zidan"]), k=1)
print(f"Top recommendations: {titles[0]}")

Top recommendations: [b'HUDwzgELY1OEzwieoHXK']


In [56]:
MAIN_TEXT_MODEL.save_weights("saved_model_weights/text_query_v1")
MAIN_USER_MODEL.save_weights("saved_model_weights/user_query_v1")
MAIN_CAREGIVER_MODEL.save_weights("saved_model_weights/caregiver_query_v1")

In [63]:
feature_layer_caregiver = None
feature_layer_user = None

from typing import Dict, Text
def build_recommend_model():
    global feature_layer_caregiver, feature_layer_user
    feature_columns = []

    number_feature = ["Age"]
    number_feature += unique_masalah_user.tolist()
    print(unique_masalah_caregiver)
    # numeric cols
    for header in number_feature:
        feature_columns.append(feature_column.numeric_column(header))

    age_feature = ["Age"]
    for col in age_feature:
        age = feature_column.numeric_column(col)
        age_buckets = feature_column.bucketized_column(age, boundaries=[17, 21, 25, 29, 33, 37, 41, 46])
        feature_columns.append(age_buckets)

    # indicator_columns
    indicator_column_names = ['Gender']
    for col_name in indicator_column_names:
      categorical_column = feature_column.categorical_column_with_vocabulary_list(
          col_name, user_dataframe[col_name].unique())
      indicator_column = feature_column.indicator_column(categorical_column)
      feature_columns.append(indicator_column)
    
    feature_layer_user= tf.keras.layers.DenseFeatures(feature_columns)
    
    feature_columns = []

    number_feature = ["Caregiver_Age"]
    number_feature += unique_masalah_caregiver
    print(unique_masalah_caregiver)
    # numeric cols

    for header in number_feature:
        feature_columns.append(feature_column.numeric_column(header))

    age_feature = ["Caregiver_Age"]
    for col in age_feature:
        age = feature_column.numeric_column(col)
        age_buckets = feature_column.bucketized_column(age, boundaries=[17, 21, 25, 29, 33, 37, 41, 46])
        feature_columns.append(age_buckets)

    col_name = 'Caregiver_Gender'
    categorical_column = feature_column.categorical_column_with_vocabulary_list(
          col_name, user_dataframe[col_name].unique())
    indicator_column = feature_column.indicator_column(categorical_column)
    feature_columns.append(indicator_column)
    
    feature_layer_caregiver= tf.keras.layers.DenseFeatures(feature_columns)
    
    

In [64]:
class RecomendModel(tfrs.Model):

  def __init__(self):
    super().__init__()

    embedding_dimension = 32

    # Set up a model for representing users.
    self.user_model = tf.keras.Sequential([
        feature_layer_user,
        layers.Dense(64),
        layers.Dense(32)
    ])

    # Set up a model for representing caregiver.
    self.caregiver_model = tf.keras.Sequential([
        feature_layer_caregiver,
        layers.Dense(64),
        layers.Dense(32)
    ])

    # Set up a task to optimize the model and compute metrics.
    self.task = tfrs.tasks.Retrieval(
      metrics=tfrs.metrics.FactorizedTopK(
        candidates=caregiver_ds.batch(5).map(self.caregiver_model)
      )
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    
    caregiver_features = ['Caregiver_Gender', 'Caregiver_Age',  'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']
    user_features = ['Gender', 'Age', 'ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan','Gangguan-stres-pascatrauma', 'Skizofrenia']
    
    
    # We pick out the user features and pass them into the user model.
    
    print("AAAAAAAAAAAAAAAAAA")
    user_embeddings = self.user_model({
        'Age':features['Age'],
        'Gender':features['Gender'],
        'ADHD-Hiperaktif-dan-kurang-fokus':features['ADHD-Hiperaktif-dan-kurang-fokus'],
        'Depresi':features['Depresi'],
        'Gangguan-kecemasan':features['Gangguan-kecemasan'],
        'Gangguan-makan':features['Gangguan-makan'],
        'Gangguan-stres-pascatrauma':features['Gangguan-stres-pascatrauma'],
        'Skizofrenia':features['Skizofrenia']
    })
    print("BBBBBBBBBBBBBB")
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.caregiver_model({
        'Caregiver_Age':features['Caregiver_Age'],
        'Caregiver_Gender':features['Caregiver_Gender'],
        'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus':features['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus'],
        'Caregiver-Depresi':features['Caregiver-Depresi'],
        'Caregiver-Gangguan-kecemasan':features['Caregiver-Gangguan-kecemasan'],
        'Caregiver-Gangguan-makan':features['Caregiver-Gangguan-makan'],
        'Caregiver-Gangguan-stres-pascatrauma':features['Caregiver-Gangguan-stres-pascatrauma'],
        'Caregiver-Skizofrenia':features['Caregiver-Skizofrenia']
        
    })
    

    # The task computes the loss and the metrics.

    return self.task(user_embeddings, positive_movie_embeddings, compute_metrics=not training)

In [65]:
build_recommend_model()

NameError: name 'unique_masalah_user' is not defined